# 🎓 Academate: AI-Powered College Assistant
**Google AI Agent Capstone Project**

In [ ]:
# 📦 Install Dependencies
# Uncomment the line below if running in a new environment (e.g., Colab)
# !pip install -r requirements.txt ipython pandas

## 📘 Project Overview
Academate is an intelligent conversational agent designed to help students navigate college life. Powered by **Gemini 2.5 Flash Lite** and the **Google Agent Development Kit (ADK)**, it provides instant answers about exam schedules, faculty details, timetables, and academic events.

This notebook demonstrates the agent's capabilities, architecture, and underlying data sources.

## ✅ Key Features
- **Exam Schedules**: Finds dates, times, and venues.
- **Previous Papers**: Retrieves links to past question papers.
- **Timetables**: Shows daily or weekly class schedules.
- **Faculty Info**: Provides contact details and office hours.
- **Academic Calendar**: Tracks holidays and deadlines.
- **Student Results**: Checks grades and SGPA.

## 💡 Usage Examples
> "When is the 3rd-year CS exam week?"
> "Share the timetable for CS2024001."
> "Give me last year's CS301 papers."
> "Who teaches Operating Systems?"

# 🏗️ System Architecture
The agent uses a deterministic tool-use pattern. Gemini acts as the reasoning engine, selecting the appropriate Python function to query CSV datasets.

In [ ]:
from IPython.display import Image, Markdown, display

display(Image(filename="assets/graph_image.png", embed=True))
display(Markdown("> The same architecture graphic is shared across the README and notebook to keep reviewers aligned."))

### 🧠 Runtime Flow

1.  **Input Processing:** The Gemini agent receives the student's natural language query.
2.  **Reasoning & Tool Selection:** The model determines if external information is needed and selects the appropriate tool (e.g., `query_exam_schedule`).
3.  **Data Retrieval:** The tool executes, querying the CSV datasets to fetch specific records.
4.  **Response Generation:** The agent synthesizes the retrieved data into a natural, helpful response for the student.

### 🔧 Tool Catalog

The agent is equipped with the following tools to handle specific student queries:

| Tool Function | Purpose | Example User Query |
| :--- | :--- | :--- |
| `query_exam_schedule` | Retrieve exam dates and times. | "When is the Python exam?" |
| `fetch_previous_papers` | Find past question papers. | "Show me last year's papers for CS101." |
| `get_class_timetable` | Get the weekly schedule for a student. | "What is my class schedule?" |
| `get_faculty_info` | Look up faculty contact details. | "What is Prof. Smith's email?" |
| `get_academic_calendar` | List upcoming academic events. | "When does the semester end?" |
| `check_student_results` | Retrieve exam results (requires ID). | "Show me my results." |

## ⚙️ Step 2: Environment Setup

The agent relies on environment variables for configuration (API keys, model settings). This cell loads the `.env` file and verifies the data directory structure.

In [ ]:
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()

DATA_DIR = Path.cwd() / "data"
csv_files = sorted(p.name for p in DATA_DIR.glob("*.csv"))

print(f"Data directory: {DATA_DIR}")
print(f"CSV files ({len(csv_files)}):")
for name in csv_files:
    print(f" - {name}")

Data directory: c:\Program Files (x86)\sahilaw\AI-Agent-Course-With-Google\Academate\data
CSV files (7):
 - academic_calendar.csv
 - exam_schedule.csv
 - faculty.csv
 - previous_papers.csv
 - student_results.csv
 - students.csv
 - timetable.csv


## 📊 Step 3: Data Preview

The agent's knowledge is grounded in CSV files located in the `data/` directory. This step previews the raw data to ensure it is loaded correctly and contains the expected information.

In [2]:
from chatbot.datasets import load_rows

def preview_dataset(name: str, limit: int = 3) -> None:
    rows = load_rows(name)
    print(f"\n{name} ({len(rows)} rows)")
    for row in rows[:limit]:
        print("  •", row)

for dataset in ["students", "exam_schedule", "timetable", "faculty", "academic_calendar", "previous_papers", "student_results"]:
    preview_dataset(dataset)


students (11 rows)
  • {'student_id': 'CS2024001', 'name': 'Rahul Sharma', 'email': 'rahul.sharma@college.edu', 'department': 'Computer Science', 'semester': '3', 'section': 'A', 'academic_year': '2024-25', 'phone': '9876543210'}
  • {'student_id': 'CS2024002', 'name': 'Priya Singh', 'email': 'priya.singh@college.edu', 'department': 'Computer Science', 'semester': '3', 'section': 'A', 'academic_year': '2024-25', 'phone': '9876543211'}
  • {'student_id': 'CS2024003', 'name': 'Sneha Gupta', 'email': 'sneha.gupta@college.edu', 'department': 'Computer Science', 'semester': '2', 'section': 'A', 'academic_year': '2024-25', 'phone': '9876543213'}

exam_schedule (13 rows)
  • {'exam_id': 'EXAM2024001', 'subject_code': 'CS301', 'subject_name': 'Data Structures and Algorithms', 'department': 'Computer Science', 'semester': '3', 'academic_year': '2024-25', 'exam_date': '2024-11-18', 'exam_time': '09:00', 'duration_minutes': '180', 'room_number': 'Hall A', 'exam_type': 'Mid-Term', 'total_marks': 

## 🛠️ Step 4: Tool Verification

Before running the full agent, we verify each tool function individually. This ensures that the logic for querying the CSVs is correct and returns the expected strings.

In [3]:
from chatbot.tools import (
    check_student_results,
    fetch_previous_papers,
    get_academic_calendar,
    get_class_timetable,
    get_faculty_info,
    query_exam_schedule,
)

examples = [
    ("Exam Schedule", lambda: query_exam_schedule("Computer Science", 3, "2024-25")),
    ("Previous Papers", lambda: fetch_previous_papers("CS301", years=3)),
    ("Timetable", lambda: get_class_timetable(student_id="CS2024001")),
    ("Faculty Info", lambda: get_faculty_info(faculty_name="Ramesh")),
    ("Academic Calendar", lambda: get_academic_calendar(days_ahead=30)),
    ("Student Results", lambda: check_student_results("CS2024001")),
]

for label, fn in examples:
    print(f"\n--- {label} ---")
    print(fn())
    print()


--- Exam Schedule ---
**Exam Schedule – Computer Science – Semester 3**
Academic Year: 2024-25

**Data Structures and Algorithms** (CS301)
   Date: 18 November 2024
   Time: 09:00
    Duration: 180 minutes
   Room: Hall A
   Type: Mid-Term
   Total Marks: 100

**Database Management Systems** (CS302)
   Date: 20 November 2024
   Time: 14:00
    Duration: 180 minutes
   Room: Hall B
   Type: Mid-Term
   Total Marks: 100

**Operating Systems** (CS303)
   Date: 23 November 2024
   Time: 09:00
    Duration: 180 minutes
   Room: Hall A
   Type: Mid-Term
   Total Marks: 100


--- Previous Papers ---
**Previous Papers – CS301**

**2023 – Mid-Term**
  Subject: Data Structures and Algorithms
  Department: Computer Science | Semester: 3
  Marks: 100
   Link: https://college.edu/papers/2023/cs301-midterm.pdf

**2023 – End-Term**
  Subject: Data Structures and Algorithms
  Department: Computer Science | Semester: 3
  Marks: 100
   Link: https://college.edu/papers/2023/cs301-endterm.pdf

**2022 – M

## 🤖 Step 5: Agent Runtime Initialization

We construct the agent runtime using the `build_agent_bundle` helper. This initializes the Gemini model, binds the tools, and prepares the execution loop.

In [4]:
from chatbot.runtime import build_agent_bundle

agent, runner = build_agent_bundle()

print(f"Agent: {agent.name}")
print(f"Model: {agent.model}")
print(f"Tools loaded: {len(agent.tools)}")

Agent: Academate
Model: model='gemini-2.5-flash-lite' speech_config=None retry_options=HttpRetryOptions(
  attempts=5,
  exp_base=7.0,
  http_status_codes=[
    429,
    500,
    503,
    504,
  ],
  initial_delay=1.0
)
Tools loaded: 6


## 🧠 Step 6: Session Memory
Academate now ships with an in-memory session store. We can persist short transcripts per session id, inspect the last few turns, and reset the context when needed.

In [1]:
from chatbot.memory import SESSION_MEMORY

session_id = "notebook-demo"
SESSION_MEMORY.clear(session_id)

demo_turns = [
    ("user", "When is the CS semester 5 exam block?"),
    ("assistant", "Mid-November, check Hall A for DSA."),
    ("user", "Who teaches Operating Systems?"),
    ("assistant", "Dr. Ramesh Verma handles OS for CS semester 3."),
]

for role, content in demo_turns:
    SESSION_MEMORY.append(session_id, role, content)

print("Recent transcript:")
print(SESSION_MEMORY.summarize(session_id))

Recent transcript:
User: When is the CS semester 5 exam block?
Assistant: Mid-November, check Hall A for DSA.
User: Who teaches Operating Systems?
Assistant: Dr. Ramesh Verma handles OS for CS semester 3.


## 📈 Step 7: Observability
We capture each CLI turn in `logs/agent.log` so it’s easy to audit which prompts ran. This snippet tails the latest lines.

In [2]:
from pathlib import Path
from chatbot.observability import get_logger

logger = get_logger()
logger.info("Notebook observability sample", extra={"session": "notebook-demo"})

log_path = Path("logs/agent.log")
if log_path.exists():
    tail = "\n".join(log_path.read_text(encoding="utf-8").splitlines()[-5:])
    print(tail)
else:
    print("No log file yet. Run the CLI once to generate logs.")

2025-11-22 11:21:11,651 - INFO - Notebook observability sample


## ✅ Step 8: Automated Evaluation
A lightweight harness runs deterministic tool calls and flags regressions if expected phrases go missing. Each case maps to a capstone requirement.

In [7]:
import importlib
from chatbot import evaluation as evaluation_module

importlib.reload(evaluation_module)
results = evaluation_module.run_evaluations()
for name, payload in results.items():
    status = "PASS" if payload["passed"] else "FAIL"
    print(f"{name}: {status}")
    print(f" Preview: {payload['preview']}")
    print()

Exam Schedule: PASS
 Preview: **Exam Schedule – Computer Science – Semester 3**
Academic Year: 2024-25

 Data Structures and Algorithms (CS301)
 Date: 18 November 2024
 Time: 09:00
 Duration: 180 minutes
 Room: Hall A
 Type: Mid-T

Timetable: PASS
 Preview: Class Timetable
Department: Computer Science | Semester: 3 | Section: A

**Monday**
  09:00 - 10:00: Data Structures and Algorithms (CS301)
     Dr. Ramesh Verma |  CS-101 | Type: Lecture

  10:00 - 1

Faculty: PASS
 Preview: Faculty Information

Dr. Ramesh Verma
 Faculty ID: FAC001
 Department: Computer Science
 Designation: Professor
 Specialization: Machine Learning, AI
   Email: ramesh.verma@college.edu
 Phone: 9876540

Academic Calendar: PASS
 Preview: No events found in the next 30 days.

Student Results: PASS
 Preview: **Student Results**
Student: Rahul Sharma (CS2024001)
Department: Computer Science

Semester 2 – 2023-24
 Programming in Java (CS201)
 Marks: 85/100 | Grade: A | Status: Pass
 Data Communications (CS2

Previous

## 💬 Step 9: Interactive Conversation

This section simulates a conversation with the agent. We iterate through a list of sample questions, and the agent processes each one, deciding whether to use a tool or answer directly.

> **Note:** Ensure `GOOGLE_API_KEY` is set in your `.env` file.

In [6]:
import asyncio

sample_questions = [
    "Hi! Who are you?",
    "When is the 5th semester CS exam block?",
    "Share the timetable for CS2024001.",
    "Any events in the next 20 days?",
]

for question in sample_questions:
    print(f"\nUser > {question}")
    await runner.run_debug(question)


User > Hi! Who are you?

 ### Created new session: debug_session_id

User > Hi! Who are you?
Academate > Hello! I'm Academate, your college helper. I can help you with exam schedules, class timetables, previous exam papers, faculty information, and academic calendar events. How can I assist you today?

User > When is the 5th semester CS exam block?

 ### Continue session: debug_session_id

User > When is the 5th semester CS exam block?
Academate > Hello! I'm Academate, your college helper. I can help you with exam schedules, class timetables, previous exam papers, faculty information, and academic calendar events. How can I assist you today?

User > When is the 5th semester CS exam block?

 ### Continue session: debug_session_id

User > When is the 5th semester CS exam block?
Academate > I can help you with that! Could you please specify the academic year for which you need the exam schedule?

User > Share the timetable for CS2024001.

 ### Continue session: debug_session_id

User > S

## 🚀 Step 10: Next Steps

- **Testing:** Run `python tests/test_agent.py` to execute the full test suite.
- **CLI Demo:** Launch the interactive CLI with `python -m chatbot.main`.
- **Web Interface:** Start the web server with `adk web .` (requires Google ADK).
- **Maintenance:** Update the CSV files in `data/` to refresh the agent's knowledge base.